In [ ]:
#!/usr/bin/env python
# Numcosmo import
try:
    import gi

    gi.require_version("NumCosmo", "1.0")
    gi.require_version("NumCosmoMath", "1.0")
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
import sys

# General imports
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
from astropy.table import Table
from astropy.io import fits
import pandas as pd

# DC2 catalog-related imports
import FoFCatalogMatching
import GCRCatalogs
from GCR import GCRQuery

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [ ]:
# A list with all DC2 catalogs available

GCRCatalogs.get_available_catalog_names(include_default_only=False)

In [ ]:
# We are loading the cosmoDC2_redmapper catalalog with richness
redmapper_cat = GCRCatalogs.load_catalog("cosmoDC2_v1.1.4_redmapper_v0.8.1")

# Taking some important information about the fiducial cosmological model and survey region
redmapper_cosmology = redmapper_cat.cosmology
sky_area = redmapper_cat.sky_area

# Listing all quantities in the choosen catalog
np.sort(redmapper_cat.list_all_quantities())

In [ ]:
z_min = 0
z_max = 1
richness_min = 0
richness_max = 200
data = redmapper_cat.get_quantities(
    ["cluster_id", "redshift", "redshift_err", "richness", "richness_err", "ra", "dec"],
    filters=[
        f"redshift > {z_min}",
        f"redshift < {z_max}",
        f"richness > {richness_min}",
        f"richness < {richness_max}",
    ],
)


redshift_err = data["redshift_err"]
richness = data["richness"]
richness_err = data["richness_err"]
redshift = data["redshift"]
ra = data["ra"]
dec = data["dec"]
cluster_id = data["cluster_id"]

ncountdata = {
    "cluster_id": cluster_id,
    "richness": richness,
    "richness_err": richness_err,
    "z": redshift,
    "z_err": redshift_err,
    "ra": ra,
    "dec": dec,
}
ncountdata = pd.DataFrame(ncountdata)


N_cl = len(ncountdata)
print(
    f"There are {N_cl} halos with Richness= [{richness_min:.2e},{richness_max:.2e}] and z=[{z_min},{z_max}] in {redmapper_cat.sky_area} deg2"
)
display(ncountdata)

# Numcosmo Cosmology

In [ ]:
cosmo = Nc.HICosmoDEXcdm()
cosmo.props.H0 = redmapper_cosmology.H0.value
cosmo.props.Omegab = redmapper_cosmology.Ob0
cosmo.props.Omegac = redmapper_cosmology.Odm0
cosmo.props.Omegax = redmapper_cosmology.Ode0

reion = Nc.HIReionCamb.new()
prim = Nc.HIPrimPowerLaw.new()

cosmo.add_submodel(reion)
cosmo.add_submodel(prim)

dist = Nc.Distance.new(2.0)

tf = Nc.TransferFunc.new_from_name("NcTransferFuncEH")

psml = Nc.PowspecMLTransfer.new(tf)
psml.require_kmin(1.0e-6)
psml.require_kmax(1.0e3)

psf = Ncm.PowspecFilter.new(psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()

mulf = Nc.MultiplicityFuncBocquet.new()
mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200.0)
mulf.set_sim(Nc.MultiplicityFuncBocquetSim.DM)

hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
hmf.set_area(sky_area)

In [ ]:
cluster_z = Nc.ClusterRedshift.new_from_name(
    "NcClusterPhotozGauss{'pz-min': <%20.15e>, 'pz-max':<%20.15e>}" % (z_min, z_max)
)
cluster_m = Nc.ClusterMass.new_from_name(
    "NcClusterMassAscaso{'M0':<%20.15e>,'z0':<%20.15e>,'lnRichness-min':<%20.15e>, 'lnRichness-max':<%20.15e>}"
    % (3e14 / (0.71), 0.6, richness_min, richness_max)
)
cluster_m.param_set_by_name("mup0", 3.19)
cluster_m.param_set_by_name("mup1", 2 / np.log(10))
cluster_m.param_set_by_name("mup2", -0.7 / np.log(10))
cluster_m.param_set_by_name("sigmap0", 0.33)
cluster_m.param_set_by_name("sigmap1", -0.08 / np.log(10))
cluster_m.param_set_by_name("sigmap2", 0 / np.log(10))

# Preparing the fit with Numcosmo

In [ ]:
ca = Nc.ClusterAbundance.new(hmf, None)
mset = Ncm.MSet.new_array([cosmo, cluster_m, cluster_z])
ncdata = Nc.DataClusterNCount.new(ca, "NcClusterPhotozGauss", "NcClusterMassAscaso")


z_obs_params = Ncm.Matrix.new(len(redshift_err), 2)
for i in range(len(redshift_err)):
    for j in range(0, 2):
        if j == 0:
            z_obs_params.set(i, j, redshift_err[i])
        if j == 1:
            z_obs_params.set(i, j, 0)

ncdata.set_lnM_obs(Ncm.Matrix.new_array(richness, 1))
ncdata.set_z_obs(Ncm.Matrix.new_array(redshift, 1))
ncdata.set_z_obs_params(z_obs_params)

ncdata.true_data(False)
ncdata.set_init(True)

# Bins definition
z_edges = np.linspace(z_min, z_max, 5)
proxy_edges = np.linspace(richness_min, richness_max, 4)

ncdata.del_bins()
for rich_l, rich_u in zip(proxy_edges, proxy_edges[1:]):
    for zl, zu in zip(z_edges, z_edges[1:]):
        lnM_obs_lb = Ncm.Vector.new_array([rich_l])
        lnM_obs_ub = Ncm.Vector.new_array([rich_u])
        z_obs_lb = Ncm.Vector.new_array([zl])
        z_obs_ub = Ncm.Vector.new_array([zu])

        ncdata.add_bin(lnM_obs_lb, lnM_obs_ub, z_obs_lb, z_obs_ub)

ncdata.bin_data()

ncdata.catalog_save("redmapper_richness_ncounts.fits", True)

In [ ]:
ncdata_fits = fits.open("redmapper_richness_ncounts.fits")
ncdata_fits.info()

ncdata_unbinned_data = ncdata_fits[1].data
ncdata_binned_data = ncdata_fits[2].data

ncdata_unbinned_Table = Table(ncdata_unbinned_data)
display(ncdata_unbinned_Table)

ncdata_binned_Table = Table(ncdata_binned_data)
display(ncdata_binned_Table)